이 자료는 위키독스 딥 러닝을 이용한 자연어 처리 입문의 코사인 유사도 튜토리얼 자료입니다.  

링크 : https://wikidocs.net/24603

# 1. 코사인 유사도(Cosine Similarity)

In [1]:
from numpy import dot
from numpy.linalg import norm
import numpy as np

In [2]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [3]:
doc1 = np.array([0,1,1,1])
doc2 = np.array([1,0,1,1])
doc3 = np.array([2,0,2,2])

In [4]:
print('문서 1과 문서2의 유사도 :',cos_sim(doc1, doc2))
print('문서 1과 문서3의 유사도 :',cos_sim(doc1, doc3))
print('문서 2와 문서3의 유사도 :',cos_sim(doc2, doc3))

문서 1과 문서2의 유사도 : 0.6666666666666667
문서 1과 문서3의 유사도 : 0.6666666666666667
문서 2와 문서3의 유사도 : 1.0000000000000002


# 2. 유사도를 이용한 추천 시스템 구현하기

In [20]:
# 판다스(Pandas) 라이브러리 불러오기
# 데이터프레임(DataFrame) 형태로 데이터 처리와 분석을 할 수 있게 해줌
import pandas as pd

# sklearn의 TF-IDF 벡터화 도구 불러오기
# TF-IDF(Term Frequency - Inverse Document Frequency)는
# 문서에서 단어의 중요도를 수치화하여 벡터로 표현해주는 기법
from sklearn.feature_extraction.text import TfidfVectorizer

# sklearn의 코사인 유사도 함수 불러오기
# 두 벡터 간의 각도를 이용해 유사도를 계산 (0~1 사이 값, 1에 가까울수록 유사)
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
# 영화 메타데이터 CSV 파일을 GitHub 원격 저장소에서 불러오기
# 'url' 변수에 CSV 파일의 경로(웹 주소)를 저장
url = 'https://raw.githubusercontent.com/sinbba77/tensorflow-nlp-tutorial/main/05.%20Vector%20Similarity/dataset/movies_metadata.csv'

# pandas의 read_csv() 함수를 사용해 CSV 파일을 읽어옴
# low_memory=False : 대용량 CSV 파일을 읽을 때 dtype 추론 오류를 줄이기 위해 설정
data = pd.read_csv(url, low_memory=False)

# 데이터프레임의 상위 2개 행 출력
# → 데이터의 구조와 컬럼을 빠르게 확인 가능
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [22]:
# 데이터프레임에서 상위 20,000개 행만 추출하여 사용
# → 전체 데이터셋이 너무 크기 때문에, 메모리와 연산 시간을 절약하기 위함
# → 실습 및 모델 학습 시 빠른 처리를 위해 데이터 샘플링 효과
data = data.head(20000)

In [23]:
# 'overview' 열의 결측값(null) 개수를 출력
# → 영화 설명(overview) 텍스트가 비어 있는 데이터가 몇 개인지 확인
# isnull() : 결측값 여부(True/False) 반환
# sum() : True 값을 모두 더해 결측값 개수를 계산
print('overview 열의 결측값의 수:', data['overview'].isnull().sum())

overview 열의 결측값의 수: 135


In [24]:
# 'overview' 열의 결측값을 빈 문자열('')로 대체
# → TF-IDF 벡터화를 적용할 때 NaN(결측값)이 있으면 오류가 발생할 수 있음
# → 따라서 결측값을 공백 문자열로 바꿔 안전하게 처리
data['overview'] = data['overview'].fillna('')

In [25]:
# 1. TF-IDF 벡터라이저 객체 생성
# stop_words='english' : 영어 불용어(예: the, is, and 등)를 자동으로 제거
tfidf = TfidfVectorizer(stop_words='english')

# 2. 'overview' 컬럼(영화 설명 텍스트)에 대해 TF-IDF 벡터화 수행
# fit_transform() : 단어 사전을 학습(fit)하고, 각 문서를 벡터로 변환(transform)
tfidf_matrix = tfidf.fit_transform(data['overview'])

# 3. TF-IDF 행렬의 크기 출력
# (문서 수, 전체 단어 개수) 형태의 행렬이 생성됨
print('TF-IDF 행렬의 크기(shape) :', tfidf_matrix.shape)

TF-IDF 행렬의 크기(shape) : (20000, 47487)


In [26]:
# 코사인 유사도(cosine similarity) 계산
# tfidf_matrix와 tfidf_matrix를 비교하여,
# 각 문서(영화 overview) 간의 유사도를 계산
# 결과: (문서 수 x 문서 수) 크기의 2차원 배열 (유사도 행렬)
# 값의 범위: 0 ~ 1 (1에 가까울수록 두 문서가 매우 유사함)
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [27]:
# 코사인 유사도 행렬의 크기를 출력
# → (문서 수, 문서 수) 형태의 2차원 배열
# → 각 문서(영화 overview)와 다른 모든 문서 간의 유사도가 저장됨
print('코사인 유사도 연산 결과 :', cosine_sim.shape)

코사인 유사도 연산 결과 : (20000, 20000)


In [28]:
# 영화 제목(title)과 데이터프레임의 인덱스를 매핑하는 딕셔너리 생성
# zip(data['title'], data.index) : 영화 제목과 그에 해당하는 행 번호(인덱스)를 묶음
# dict(...) : (제목 → 인덱스) 형태의 딕셔너리로 변환
# 예) {'Toy Story': 0, 'Jumanji': 1, 'Grumpier Old Men': 2, ...}
title_to_index = dict(zip(data['title'], data.index))

In [29]:
# 'Father of the Bride Part II'라는 영화 제목에 해당하는 인덱스를 가져옴
# title_to_index 딕셔너리를 이용해 (제목 → 인덱스) 매핑된 값을 조회
idx = title_to_index['Father of the Bride Part II']

# 해당 영화의 인덱스 출력
print(idx)

4


In [18]:
# 영화 추천 함수 정의
# 입력: 영화 제목(title), 유사도 행렬(cosine_sim)
# 출력: 입력한 영화와 가장 유사한 영화 10편의 제목 리스트
def get_recommendations(title, cosine_sim=cosine_sim):
    # 1. 입력받은 영화 제목의 인덱스 찾기
    idx = title_to_index[title]

    # 2. 해당 영화와 다른 모든 영화와의 유사도 점수 가져오기
    # enumerate() → (영화 인덱스, 유사도 점수) 튜플 리스트 생성
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 3. 유사도 점수를 기준으로 내림차순 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 4. 자기 자신을 제외하고 상위 10개만 추출
    sim_scores = sim_scores[1:11]

    # 5. 추천할 영화들의 인덱스만 추출
    movie_indices = [idx[0] for idx in sim_scores]

    # 6. 해당 인덱스에 해당하는 영화 제목 반환
    return data['title'].iloc[movie_indices]

In [19]:
# 'The Dark Knight Rises'와 가장 유사한 영화 10편 추천
print(get_recommendations('The Dark Knight Rises'))

,title
12481,The Dark Knight
150,Batman Forever
1328,Batman Returns
15511,Batman: Under the Red Hood
585,Batman
9230,Batman Beyond: Return of the Joker
18035,Batman: Year One
19792,"Batman: The Dark Knight Returns, Part 1"
3095,Batman: Mask of the Phantasm
10122,Batman Begins
